In [56]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import copy
import pickle as pkl
import itertools
from difflib import SequenceMatcher
from difflib import get_close_matches
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

In [57]:
mehakInput_List1 = [['Burgers, Mocktails, Pasta, Cocktails, Beef Steak, Lunch Buffet, Chicken Steak'], ['Kamikaze, Long Island Iced Tea, Pizza, Hefeweizen, Cocktails, Belgian Brew Beer, Chicken Bbq Wings']]
mehakInput_List2 = [['English Breakfast, Sandwiches, Chicken Burger, Pasta, Veg Burger, Chicken Sausages'], ['English Breakfast, French Fries, Burgers, Pancakes, Corn Sandwich, Chicken Burger, Bacon Fry']]
mehakInput_List3 = [['Masala Fries, Chicken Wings, Beer, Onion Rings, Pizza, Nachos']]

In [58]:
def getMaxMatch(nameList,inputList,count,finalProcessedRecipeNameAndIngredients):

    x = 0
    result = dict()
    for i in nameList:

        similar_sentence = None
        similar_sentence_index = -1
        similar_sentence = process.extractOne(i,inputList,scorer=fuzz.partial_token_sort_ratio)

        if similar_sentence is not None:
            similar_sentence_ingredients = finalProcessedRecipeNameAndIngredients[similar_sentence[0]]
            result[similar_sentence[0]] = similar_sentence_ingredients
            print("******************")
            print(similar_sentence)
            print("This is similiar to original sentence ")
            print("original sentence")
            print(i)
            print("It's ingredients are")
            print(similar_sentence_ingredients)
            print("******************** \n")
            x += 1
            if x == count:
                break
    return result

In [59]:
def getTokens(mehakOutput):
    mehakOutputTokens = []

    for i in range(len(mehakOutput)):
        a = mehakOutput[i][0].split(', ')
        mehakOutputTokens = mehakOutputTokens + a

    return mehakOutputTokens

In [60]:
recipes_RAW = pd.read_csv("RAW_recipes.csv")
recipes_RAW = recipes_RAW.rename({"id": "recipe_id"}, axis=1)

recipes_RAW_name = recipes_RAW["name"]
recipes_RAW_name = [str(i) for i in recipes_RAW_name]

recipes_RAW_ingredients = recipes_RAW["ingredients"]
recipes_RAW_ingredients = [str(i) for i in recipes_RAW_ingredients]

In [61]:
finalProcessedRecipeNameAndIngredients = dict()
for i in range(len(recipes_RAW_name)):
    temp = recipes_RAW_name[i]
    finalProcessedRecipeNameAndIngredients[temp] = recipes_RAW_ingredients[i]
    
for key in finalProcessedRecipeNameAndIngredients.keys():
    print(key)
    print("*************")
    print(finalProcessedRecipeNameAndIngredients[key])
    break

arriba   baked winter squash mexican style
*************
['winter squash', 'mexican seasoning', 'mixed spice', 'honey', 'butter', 'olive oil', 'salt']


In [62]:
mehakInput_List1_Tokens = getTokens(mehakInput_List1)
count_List1 = 2
mehakInput_List2_Tokens = getTokens(mehakInput_List2)
count_List2 = 2
mehakInput_List3_Tokens = getTokens(mehakInput_List3)
count_List3= 1

In [63]:
result_List1 = getMaxMatch(mehakInput_List1_Tokens,recipes_RAW_name,count_List1,finalProcessedRecipeNameAndIngredients)
result_List2 = getMaxMatch(mehakInput_List2_Tokens,recipes_RAW_name,count_List2,finalProcessedRecipeNameAndIngredients)
result_List3 = getMaxMatch(mehakInput_List3_Tokens,recipes_RAW_name,count_List3,finalProcessedRecipeNameAndIngredients)

print(result_List1)
print(result_List2)
print(result_List3)

******************
('grilled  venison burgers', 100)
This is similiar to original sentence 
original sentence
Burgers
It's ingredients are
['ground venison', 'egg substitute', 'non-fat powdered milk', 'water', 'fresh breadcrumb', 'onion', 'salt', 'black pepper', 'dry mustard', 'worcestershire sauce']
******************** 

******************
('grandparent mocktails  1 and  2  non alcoholic', 100)
This is similiar to original sentence 
original sentence
Mocktails
It's ingredients are
['cola', 'grenadine', 'orange slice', 'maraschino cherry', 'frozen orange juice concentrate', 'milk', 'water', 'sugar', 'vanilla']
******************** 

******************
('breakfast apple english muffin rounds', 100)
This is similiar to original sentence 
original sentence
English Breakfast
It's ingredients are
['peanut butter', 'english muffins', 'red apple', 'brown sugar', 'butter', 'cinnamon']
******************** 

******************
('geebee special  sandwiches', 100)
This is similiar to original se

In [64]:
resultRecipe = set()
resultIngredients = set()

for key, value in result_List1.items():
    resultRecipe.add(key)
    value = value[1:]
    value = value[:-1]
    temp = value.split(', ')
    resultIngredients.update(temp)
        
for key, value in result_List2.items():
    resultRecipe.add(key)
    value = value[1:]
    value = value[:-1]
    temp = value.split(', ')
    resultIngredients.update(temp)
        
for key, value in result_List3.items():
    resultRecipe.add(key)
    value = value[1:]
    value = value[:-1]
    temp = value.split(', ')
    resultIngredients.update(temp)

print(resultRecipe)
print("**********")
print(resultIngredients)

{'breakfast apple english muffin rounds', 'grandparent mocktails  1 and  2  non alcoholic', 'geebee special  sandwiches', 'grilled  venison burgers', 'salad'}
**********
{"'salt'", "'fresh breadcrumb'", "'maraschino cherry'", "'grenadine'", "'tomatoes'", "'milk'", "'ketchup'", "'worcestershire sauce'", "'non-fat powdered milk'", "'dry mustard'", "'onion'", "'frozen orange juice concentrate'", "'english muffins'", "'crouton'", "'american cheese'", "'sugar'", "'cucumber'", "'ground venison'", "'peanut butter'", "'butter'", "'hamburger buns'", "'orange slice'", "'ham'", "'vanilla'", "'red apple'", "'sweet pickle relish'", "'cinnamon'", "'iceberg lettuce'", "'egg substitute'", "'cola'", "'brown sugar'", "'sunflower seeds'", "'water'", "'black pepper'"}


In [73]:
similarDishes = dict()
x = 0
for i in range(len(recipes_RAW_ingredients)):
    if recipes_RAW_name[i] not in resultRecipe:
        value = recipes_RAW_ingredients[i][1:]
        value = value[:-1]
        temp = value.split(', ')
        temp = set(temp)
        if temp.issubset(resultIngredients):
            similarDishes[recipes_RAW_name[i]] = recipes_RAW_ingredients[i]
            resultRecipe.add(recipes_RAW_name[i])
            x += 1
            if x == 5:
                break

In [74]:
print(similarDishes)

{'peggy s hot peanut butter topping for ice cream': "['peanut butter', 'brown sugar']", 'pork roll   cheese sandwiches': "['ham', 'american cheese', 'hamburger buns']", 'roasted salted sunflower seed': "['sunflower seeds', 'water', 'salt']", 'roy rogers': "['grenadine', 'cola', 'maraschino cherry', 'orange slice']", 'salted boiling water   what does it mean': "['water', 'salt']"}
